In [3]:
import torch
print(torch.__version__)

OSError: [WinError 182] The operating system cannot run %1. Error loading "c:\Users\SAnuv\anaconda3\envs\ahead\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [2]:
import cv2
import numpy as np
from mtcnn import MTCNN
from deep_sort_realtime.deepsort_tracker import DeepSort
import face_recognition
import os
import pickle

In [ ]:
# Initialize DeepSORT tracker
tracker = DeepSort(max_age=10)

In [ ]:
# Load pre-trained MTCNN model for face detection
mtcnn = MTCNN()

In [ ]:
# Open video file
video_path = "Class Room Entrance.mp4"
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [ ]:
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Use "mp4v" for mp4 format
output_path = "output_video.mp4"
out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame_width, frame_height))

In [ ]:
frame_id = 0
skip_frames = 2  # Skip the frames

In [ ]:
# Size threshold for faces (adjust as needed)
min_face_size = 10000  # For example, minimum area of 10000 pixels

In [ ]:
# Dictionary to track whether the face image and embedding have been saved for each ID
saved_data = {}

In [ ]:
# Create a directory to save face images
os.makedirs("faces(V3)", exist_ok=True)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    # Skip frames until reaching the nth frame
    if frame_id <= skip_frames:
        continue

    # Detect faces using MTCNN
    faces = mtcnn.detect_faces(frame)
    bbs = [(face['box'], face['confidence'], face['keypoints']) for face in faces]

    # Update tracker with the detected faces
    tracks = tracker.update_tracks(bbs, frame=frame)

    # Draw bounding boxes and IDs on the frame
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id

        # Check if data for this ID has already been saved
        if track_id in saved_data:
            continue

        ltrb = track.to_ltrb()

        # Check face size
        face_area = (ltrb[2] - ltrb[0]) * (ltrb[3] - ltrb[1])
        if face_area < min_face_size:
            continue  # Skip small faces

        # Draw bounding box
        cv2.rectangle(frame, (int(ltrb[0]), int(ltrb[1])), (int(ltrb[2]), int(ltrb[3])), (0, 255, 0), 2)

        # Get facial landmarks using face_recognition library
        face_locations = [(int(ltrb[1]), int(ltrb[2]), int(ltrb[3]), int(ltrb[0]))]  # (top, right, bottom, left)
        landmarks = face_recognition.face_landmarks(frame, face_locations)

        if landmarks:
            # Get face embedding
            face_encoding = face_recognition.face_encodings(frame, face_locations)[0]

            # Check if the face embedding is not empty before saving
            if len(face_encoding) > 0:
                # Compare with existing embeddings
                match = False
                for saved_id, saved_info in saved_data.items():
                    saved_embedding = saved_info["embedding"]
                    # Compare the face embeddings using face_recognition library
                    results = face_recognition.compare_faces([saved_embedding], face_encoding)

                    # Check if there is a match
                    if any(results):
                        match = True
                        break

                # If no match, save the face embedding
                if not match:
                    # Save the face embedding
                    saved_data[track_id] = {
                        "embedding": face_encoding,
                        "image_path": f"faces(V3)/{track_id}_frame{frame_id}.png"
                    }

                    # Get the face bounding box coordinates
                    top, right, bottom, left = int(ltrb[1]), int(ltrb[2]), int(ltrb[3]), int(ltrb[0])

                    # Check if the bounding box coordinates are within the frame boundaries
                    if 0 <= top < frame.shape[0] and 0 <= left < frame.shape[1] and top < bottom and left < right:
                        # Capture the face image
                        face_image = frame[top:bottom, left:right]

                        # Check if the face image is not empty before saving
                        if not face_image.size == 0:
                            # Save the face image
                            cv2.imwrite(saved_data[track_id]["image_path"], face_image)

                            # Draw label
                            label = f"ID: {track_id}"
                            cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write the frame with bounding boxes to the output video
    out.write(frame)

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
saved_data